In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# Базовый URL сайта
base_url = "http://ru-pets.ru/index.php?m=6&c=2&to=1"

# Подготавливаем CSV-файл
with open('exhibitions.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Дата проведения', 'Название выставки', 'Клуб-организатор']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Проходим по всем страницам (77 страниц)
    for page in range(1, 78):
        # Формируем URL для каждой страницы
        page_url = f"{base_url}&page={page}"

        # Отправляем запрос на сайт
        response = requests.get(page_url)

        # Проверяем успешность запроса
        if response.status_code == 200:
            # Разбираем содержимое веб-страницы
            soup = BeautifulSoup(response.content, 'html.parser')

            # Находим контейнер, содержащий данные о выставках
            exhibitions = soup.find_all('div', class_='listitem')

            # Проходим по каждой выставке и извлекаем данные
            for exhibition in exhibitions:
                # Извлекаем дату
                date = exhibition.find('h2').text.strip().split(",")[0].strip()

                # Извлекаем название выставки
                name = exhibition.find('span', class_='cl-green').text.strip()

                # Извлекаем информацию о клубе-организаторе
                club_info = exhibition.find('div', class_='msgtext').text.strip()
                if "Клуб - Организатор:" in club_info:
                    club = club_info.split("Клуб - Организатор:")[1].split("\n")[0].strip()
                else:
                    club = "Не указано"

                writer.writerow({'Дата проведения': date, 'Название выставки': name, 'Клуб-организатор': club})

            print(f"Данные со страницы {page} успешно обработаны")
        else:
            print(f"Не удалось загрузить веб-страницу {page}. Код состояния: {response.status_code}")

print("Данные успешно сохранены в exhibitions.csv")